# <img align="left" src="./images/film_strip_vertical.png"     style=" width:40px;  " > Practice lab: Deep Learning for Content-Based Filtering

In this exercise, you will implement content-based filtering using a neural network to build a recommender system for movies. 


# Outline
- [ 1 - Packages ](#1)
- [ 2 - Movie ratings dataset ](#2)
- [ 3 - Content-based filtering with a neural network](#3)
  - [ 3.1 Training Data](#3.1)
  - [ 3.2 Preparing the training data](#3.2)
- [ 4 - Neural Network for content-based filtering](#4)
  - [ Exercise 1](#ex01)
- [ 5 - Predictions](#5)
  - [ 5.1 - Predictions for a new user](#5.1)
  - [ 5.2 - Predictions for an existing user.](#5.2)
  - [ 5.3 - Finding Similar Items](#5.3)
    - [ Exercise 2](#ex02)
- [ 6 - Congratulations! ](#6)


_**NOTE:** To prevent errors from the autograder, you are not allowed to edit or delete non-graded cells in this lab. Please also refrain from adding any new cells. 
**Once you have passed this assignment** and want to experiment with any of the non-graded code, you may follow the instructions at the bottom of this notebook._

<a name="1"></a>
## 1 - Packages <img align="left" src="./images/movie_camera.png"     style=" width:40px;  ">
We will use familiar packages, NumPy, TensorFlow and helpful routines from [scikit-learn](https://scikit-learn.org/stable/). We will also use [tabulate](https://pypi.org/project/tabulate/) to neatly print tables and [Pandas](https://pandas.pydata.org/) to organize tabular data.

In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
from recsysNN_utils import *
pd.set_option("display.precision", 1)

<a name="2"></a>
## 2 - Course ratings dataset <img align="left" src="./images/film_rating.png" style=" width:40px;" >
The data set is created ON OUR OWN using different sources from coursera, kaggle, udemy, edX, Linkedln,etc.  


This dataset has $n_u = 397$ users, $n_m= 847$ online courses and 25521 ratings. For each courses, the dataset provides a course title, release date, and one or more genres. For example ""Digital Electronics - RoboGrok Automation" by Terry Sturtevant" has several genres: "Arts-And-Humanities|Language-Learning|Electrical-And-Mechanical-Engineering". This dataset contains little information about users other than their ratings. This dataset is used to create training vectors for the neural networks described below. 


In [2]:
# Load Data, set configuration variables
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre = load_data()

num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items
print(f"Number of training vectors: {len(item_train)}")

Skipping line 847 due to insufficient columns.
Number of training vectors: 50884


Let's look at the first few entries in the user training array.

In [3]:
pprint_train(user_train, user_features, uvs,  u_s, maxcount=5)

[user id],[rating count],[rating ave],Course-D ifficulty,Business-An d-Management,Des ign,Environmen tal-Science,Arts-And- Humanities,Computer -Science,Data-S cience,Math-An d-Logic,Language -Learning,Hea lth,Social- Science,Electrical-And-Mec hanical-Engineering,Digital- Marketing,Physics-An d-Astronomy
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9


In [4]:
pprint_train(item_train, item_features, ivs, i_s, maxcount=5, user=False)

[course id],year,ave rating,Course-D ifficulty,Business-An d-Management,Des ign,Environmen tal-Science,Arts-And- Humanities,Computer -Science,Data-S cience,Math-An d-Logic,Language -Learning,Hea lth,Social- Science,Electrical-And-Mec hanical-Engineering,Digital- Marketing,Physics-An d-Astronomy
6874,2003,4.0,1,0,0,0,0,1,0,0,0,0,0,0,0,1
8798,2004,3.8,1,0,0,0,0,1,0,1,0,0,0,0,0,1
46970,2006,3.2,1,0,0,0,1,0,0,0,0,0,0,0,0,0
48516,2006,4.3,0,0,0,0,0,1,0,1,0,0,0,0,0,1
58559,2008,4.2,1,0,0,0,0,1,0,1,0,0,0,0,0,0


Above, the course array contains the year the course was released, the average rating and an indicator for each potential genre. The indicator is one for each genre that applies to the course. The course id is not used in training but is useful when interpreting the data.

In [5]:
print(f"y_train[:5]: {y_train[:5]}")

y_train[:5]: [4.  3.5 4.  4.  4.5]


The target, y, is the movie rating given by the user. 

In [6]:
# scale training data
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))
#ynorm_test = scalerTarget.transform(y_test.reshape(-1, 1))


True
True


To allow us to evaluate the results, we will split the data into training and test sets as was discussed in Course 2, Week 3. Here we will use [sklean train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split and shuffle the data. Note that setting the initial random state to the same value ensures item, user, and y are shuffled identically.

In [7]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test data shape: {item_test.shape}")

movie/item training data shape: (40707, 17)
movie/item test data shape: (10177, 17)


The scaled, shuffled data now has a mean of zero.

In [8]:
pprint_train(user_train, user_features, uvs, u_s, maxcount=5)

[user id],[rating count],[rating ave],Course-D ifficulty,Business-An d-Management,Des ign,Environmen tal-Science,Arts-And- Humanities,Computer -Science,Data-S cience,Math-An d-Logic,Language -Learning,Hea lth,Social- Science,Electrical-And-Mec hanical-Engineering,Digital- Marketing,Physics-An d-Astronomy
1,0,-1.0,-0.8,-0.7,0.1,-0.0,-1.2,-0.4,0.6,-0.5,-0.5,-0.1,-0.6,-0.6,-0.7,-0.7
0,1,-0.7,-0.5,-0.7,-0.1,-0.2,-0.6,-0.2,0.7,-0.5,-0.8,0.1,-0.0,-0.6,-0.5,-0.4
-1,-1,-0.2,0.3,-0.4,0.4,0.5,1.0,0.6,-1.2,-0.3,-0.6,-2.3,-0.1,0.0,0.4,-0.0
0,-1,0.6,0.5,0.5,0.2,0.6,-0.1,0.5,-1.2,0.9,1.2,-2.3,-0.1,0.0,0.2,0.3
-1,0,0.7,0.6,0.5,0.3,0.5,0.4,0.6,1.0,0.6,0.3,0.8,0.8,0.4,0.7,0.7


<a name="4"></a>
## 4 - Neural Network for content-based filtering



In [9]:
# GRADED_CELL
# UNQ_C1

num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
    #tf.keras.layers.Dense(512, activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    #tf.keras.layers.Dropout(0.5)
    tf.keras.layers.Dense(128, activation='relu'),
    
    tf.keras.layers.Dense(num_outputs),
  
  
    ### END CODE HERE ###  
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
    #tf.keras.layers.Dense(512, activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    #tf.keras.layers.Dropout(0.5)
    tf.keras.layers.Dense(128, activation='relu'),
    
    tf.keras.layers.Dense(num_outputs),
  
    ### END CODE HERE ###  
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 14)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 16)]                 0         []                            
                                                                                                  
 sequential (Sequential)     (None, 32)                   40864     ['input_1[0][0]']             
                                                                                                  
 sequential_1 (Sequential)   (None, 32)                   41376     ['input_2[0][0]']             
                                                                                              

<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>
    
  You can create a dense layer with a relu activation as shown.
    
```python     
user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
  tf.keras.layers.Dense(256, activation='relu'),

    
    ### END CODE HERE ###  
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
  tf.keras.layers.Dense(256, activation='relu'),

    
    ### END CODE HERE ###  
])
```    
<details>
    <summary><font size="2" color="darkblue"><b> Click for solution</b></font></summary>
    
```python 
user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_outputs),
    ### END CODE HERE ###  
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_outputs),
    ### END CODE HERE ###  
])
```
</details>
</details>

    


We will use a mean squared error loss and an Adam optimizer.

In [10]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [11]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=100)

Epoch 1/100


1273/1273 [==============================] - 3s 2ms/step - loss: 0.1238
Epoch 2/100
1273/1273 [==============================] - 3s 2ms/step - loss: 0.1147
Epoch 3/100
1273/1273 [==============================] - 3s 2ms/step - loss: 0.1106
Epoch 4/100
1273/1273 [==============================] - 2s 2ms/step - loss: 0.1057
Epoch 5/100
1273/1273 [==============================] - 2s 2ms/step - loss: 0.1026
Epoch 6/100
1273/1273 [==============================] - 2s 2ms/step - loss: 0.0996
Epoch 7/100
1273/1273 [==============================] - 2s 2ms/step - loss: 0.0972
Epoch 8/100
1273/1273 [==============================] - 3s 2ms/step - loss: 0.0953
Epoch 9/100
1273/1273 [==============================] - 2s 2ms/step - loss: 0.0933
Epoch 10/100
1273/1273 [==============================] - 2s 2ms/step - loss: 0.0920
Epoch 11/100
1273/1273 [==============================] - 2s 2ms/step - loss: 0.0901
Epoch 12/100
1273/1273 [==============================] - 2s 2ms/step - loss: 0.0882
E

Evaluate the model to determine loss on the test data. 

In [12]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

319/319 [==============================] - 1s 1ms/step - loss: 0.0767


0.07674766331911087

In [13]:
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

It is comparable to the training loss indicating the model has not substantially overfit the training data.

<a name="5"></a>
## 5 - Predictions
Below, you'll use your model to make predictions in a number of circumstances. 
<a name="5.1"></a>
### 5.1 - Predictions for a new user
First, we'll create a new user and have the model suggest courses for that user. After you have tried this on the example user content, feel free to change the user content to match your own preferences and see what the model suggests. Note that ratings are between 0.5 and 5.0, inclusive, in half-step increments.

In [14]:
new_user_id = 5000
new_rating_ave = 0.0
new_Course_Difficulty = 0.0
new_Business_And_Management = 5.0
new_Design = 0.0
new_Environmental_Science = 0.0
new_Arts_And_Humanities = 5.0
new_Computer_Science = 0.0
new_Data_Science = 0.0
new_Math_And_Logic = 0.0
new_Language_Learning = 5.0
new_Health = 0.0
new_Social_science = 0.0
new_Electrical_And_Mechanical_Engineering = 0.0
new_Digital_Marketing = 0.0
new_Physics_And_Astronomy = 0.0
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_Course_Difficulty, new_Business_And_Management, new_Design, new_Environmental_Science,
                      new_Arts_And_Humanities, new_Computer_Science, new_Data_Science,
                      new_Math_And_Logic, new_Language_Learning, new_Health, new_Social_science ,
                      new_Electrical_And_Mechanical_Engineering, new_Digital_Marketing, new_Physics_And_Astronomy]])

In [44]:
# generate and replicate the user vector to match the number movies in the data set.
user_vecs = gen_user_vecs(user_vec,len(item_vecs))

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction 
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #using unscaled vectors for display

table=print_pred_movies(sorted_ypu, sorted_items, movie_dict, maxcount = 10)

27/27 [==============================] - 0s 3ms/step


In [45]:
with open('output.html', 'w') as file:
    file.write(table)

<a name="5.2"></a>
### 5.2 - Predictions for an existing user.
Let's look at the predictions for "user 2", one of the users in the data set. We can compare the predicted ratings with the course's ratings.

In [43]:
uid = 2 
# form a set of user vectors. This is the same vector, transformed and repeated.
user_vecs, y_vecs = get_user_vecs(uid, user_train_unscaled, item_vecs, user_to_genre)

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction 
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #using unscaled vectors for display
sorted_user  = user_vecs[sorted_index]
sorted_y     = y_vecs[sorted_index]

#print sorted predictions for movies rated by the user
print_existing_user(sorted_ypu, sorted_y.reshape(-1,1), sorted_user, sorted_items, ivs, uvs, movie_dict, maxcount = 50)

27/27 [==============================] - 0s 2ms/step


y_p,y,user,user genre ave,movie rating ave,movie id,title,genres
4.6,5.0,2,[4.0],4.3,80906,"Data Science Specialization 2"" by the Case Western Reserve University """,Data-Science
4.4,5.0,2,"[4.0,4.1,4.0]",3.9,106782,Advanced MATLAB: Data Types and Time Series Analysis 2 - Coursera,Arts-And-Humanities|Computer-Science|Math-And-Logic
4.4,3.5,2,"[4.0,3.9,3.9]",3.9,115713,Calculus for Machine Learning and Data Science 1 - Coursera,Math-And-Logic|Digital-Marketing|Physics-And-Astronomy
4.4,5.0,2,"[4.0,4.2,3.9,3.9]",3.8,122882,Cosmic Microwave Background 2 - Universitat Autònoma de Barcelona,Course-Difficulty|Business-And-Management|Digital-Marketing|Physics-And-Astronomy
4.3,3.5,2,"[4.0,4.0]",3.9,99114,"Introduction to Mathematical Thinking"" by Stanford University""",Course-Difficulty|Math-And-Logic
4.3,4.0,2,"[4.0,4.1,4.0,4.0,3.9,3.9]",4.1,79132,Super-Earths and Life 2 - Harvard University,Course-Difficulty|Computer-Science|Math-And-Logic|Social-Science|Digital-Marketing|Physics-And-Astronomy
4.2,5.0,2,[4.0],3.6,60756,Muslim World - Villanova University,Arts-And-Humanities
4.2,3.5,2,"[4.0,4.2,4.1]",4.0,91529,Business Strategies for Social Impact 3 - University of Pennsylvania,Course-Difficulty|Business-And-Management|Computer-Science
4.2,4.5,2,"[4.0,4.1,4.0]",4.2,58559,Calculus: Single Variable 2 - University of Pennsylvania,Course-Difficulty|Computer-Science|Math-And-Logic
4.1,4.0,2,"[4.1,4.0,3.9]",4.3,48516,Quantum Physics I and Quantum Physics II - Havard,Computer-Science|Math-And-Logic|Physics-And-Astronomy


The model prediction is generally within 1 of the actual rating though it is not a very accurate predictor of how a user rates specific course. This is especially true if the user rating is significantly different than the user's genre average. You can vary the user id above to try different users. Not all user id's were used in the training set.